### Central heating control

Uses a button to run it for a number of minutes, and an MQTT channels

Run the heating for 5 seconds:
> mosquitto_pub -h 192.168.43.1 -t "esp8266H/cenheat/secs/cmd" -m "5"




Instructions from:
    https://www.esp8266.com/wiki/doku.php?id=toolchain

Make the esp C-compiler
> conda deactivate
> git clone --recursive https://github.com/pfalcon/esp-open-sdk
> cd esp-open-sdk
> make

Copy across the modules
> cp /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py /home/julian/extrepositories/micropython/ports/esp8266/modules/

> cp /home/julian/extrepositories/micropython-lib/uasyncio/uasyncio/ /home/julian/extrepositories/micropython/ports/esp8266/modules/

> cp /home/julian/extrepositories/micropython-lib/uasyncio.core/uasyncio/core.py /home/julian/extrepositories/micropython/ports/esp8266/modules/uasyncio

> cp /home/julian/extrepositories/micropython-lib/uasyncio.queues/uasyncio/queues.py /home/julian/extrepositories/micropython/ports/esp8266/modules/uasyncio

> export PATH="/home/julian/extrepositories/esp-open-sdk/xtensa-lx106-elf/bin/:$PATH"

Make the file
> cd /home/julian/extrepositories/micropython/ports/esp8266

> make


/home/julian/extrepositories/micropython/ports/esp8266/build-GENERIC/firmware-combined.bin


In [ ]:
# Now flash the built firmware withe mqtt_as and asyncio
%esptool esp8266 /home/julian/extrepositories/micropython/ports/esp8266/build-GENERIC/firmware-combined.bin

In [ ]:
# Small ESP8266

#      RST          |  GPIO1   TX
#       A0          |  GPIO3   RX
#       D0  GPIO16  |  GPIO5   D1  SCL
# SCK   D5  GPIO14  |  GPIO4   D2  SDA
# MISO  D6  GPIO12  |  GPIO0   D3
# MOSI  D7  GPIO13  |  GPIO2   D4  LED
# SS    D8  GPIO15  |          GND
#      3V3          |          5V

# (not able to get the mDNS to work for esp8266)

In [34]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [51]:
%sendtofile config.txt

wifiname     BV6000
wifipassword bubblino
mqttbroker   192.168.43.1
pinled       2
pincontrol   5
pinbuttonled 4
pinbutton    0
boardname    esp8266H

Sent 8 lines (149 bytes) to config.txt.


In [70]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [71]:
%sendtofile main.py

from mqtt_as import config, MQTTClient
import uasyncio as asyncio
import time
from machine import Pin, PWM

fconfig = dict(x.split()  for x in open("config.txt"))

pinled = Pin(int(fconfig["pinled"]), Pin.OUT)
pincontrol = Pin(int(fconfig["pincontrol"]), Pin.OUT)
pincontrol.value(0)
pinbuttonled = Pin(int(fconfig["pinbuttonled"]), Pin.OUT)
pinbutton = Pin(int(fconfig["pinbutton"]), Pin.IN, Pin.PULL_UP)
pwmbuttonled = machine.PWM(pinbuttonled, duty=5)
pinled.value(0)

config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]
config['mqttchannel'] = fconfig["boardname"]
        
boardname = fconfig["boardname"].encode()    
tcenheatsecs = boardname+b'/cenheat/secs'
tcenheatcmd = tcenheatsecs+b'/cmd'
tcenheatrep = tcenheatsecs+b'/rep'
tbuttonsecs = boardname+b'/button/secs'

nextcontrolsecson = None
buttoncontrolonsecs = 10
secondscountdown = 0

def callback(topic, msg, retained):
    global nextcontrolsecson
    print("callback", topic, msg, retained)
    if topic == tcenheatcmd:
        try:
            nextcontrolsecson = max(0, int(msg))
        except ValueError as e:
            print(e)
            
config['subs_cb'] = callback
client = MQTTClient(config)

async def controltask():
    global nextcontrolsecson, secondscountdown
    tickstimeoff = 0
    i = 0
    while True:
        await asyncio.sleep_ms(200)
        i += 1
        t0 = time.ticks_ms()
        if nextcontrolsecson is not None:
            pincontrol.value(1 if nextcontrolsecson != 0 else 0)
            tickstimeoff = t0 + nextcontrolsecson*1000
            nextcontrolsecson = None
        if tickstimeoff != 0:
            if tickstimeoff < t0:
                pincontrol.value(0)
                tickstimeoff = 0
        secondscountdown = ((tickstimeoff-t0)/1000) if tickstimeoff else 0
        if (i%10) == 0:
            await client.publish(tcenheatrep, str(secondscountdown))

async def buttontask():
    global nextcontrolsecson
    i = 0
    while True:
        await asyncio.sleep_ms(100)
        i += 1
        if pinbutton.value() == 0:
            t0 = time.ticks_ms()
            dt = 0
            while pinbutton.value() == 0:
                dt = time.ticks_ms() - t0
                if dt > 3000:
                    pwmbuttonled.duty((((dt-3000)//500)%2)*500)
                await asyncio.sleep_ms(50)
            print("buttondown", dt)
            nextcontrolsecson = (buttoncontrolonsecs  if dt < 3000  else 0)
            await client.publish(tbuttonsecs, str(dt/1000))
            i = 0

        if (i%10) == 0:
            pwmbuttonled.duty(int(min(secondscountdown/buttoncontrolonsecs, 1)*300))

        if client.isconnected():
            pinled.value(0 if (i%50) < 2 else 1)
        else:
            pinled.value(0 if (i%20) < 7 else 1)
        
async def startuptask():
    await client.connect()
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.subscribe(tcenheatcmd)
    print("subscribing to", tcenheatcmd)
    await client.publish(boardname+b'/ip', ipnumber, retain=True, qos=1)

aloop = asyncio.get_event_loop()
aloop.create_task(startuptask())
aloop.create_task(controltask())
aloop.create_task(buttontask())
aloop.run_forever()



Sent 103 lines (3271 bytes) to main.py.
